In [1]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [2]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
##종웅님 
headers = {"X-Naver-Client-Id": "EmdlUXsc4CuKNm_kxYKS",
           "X-Naver-Client-Secret":"dSLoIEmMgy"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [ ]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "남성화장품", "param": [ "50000202"]},
          {"name": "네일케어", "param": [ "50000196"]},
          {"name": "바디케어", "param": [ "50000197"]},
          {"name": "베이스메이크업", "param": [ "50000194"]},
          {"name": "뷰티소품", "param": [ "50000201"]},
          {"name": "색조메이크업", "param": [ "50000195"]},
          {"name": "선케어", "param": [ "50000191"]},
          {"name": "클렌징", "param": [ "50000192"]},
          {"name": "향수", "param": [ "50000200"]},
          {"name": "헤어케어", "param": [ "50000198"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "화장품/미용", "param": [ "50000002"]},
      {"name": "스킨케어", "param": [ "50000190"]},
    ],
    "device":i,
    "gender":"",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "화장품/미용")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "스킨케어")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["화장품/미용"], axis=1, inplace=True)
df_all_a

In [ ]:
for i in devices:
    categories = [
          {"name": "남성화장품", "param": [ "50000202"]},
          {"name": "네일케어", "param": [ "50000196"]},
          {"name": "바디케어", "param": [ "50000197"]},
          {"name": "베이스메이크업", "param": [ "50000194"]},
          {"name": "뷰티소품", "param": [ "50000201"]},
          {"name": "색조메이크업", "param": [ "50000195"]},
          {"name": "선케어", "param": [ "50000191"]},
          {"name": "클렌징", "param": [ "50000192"]},
          {"name": "향수", "param": [ "50000200"]},
          {"name": "헤어케어", "param": [ "50000198"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "화장품/미용", "param": [ "50000002"]},
      {"name": "스킨케어", "param": [ "50000190"]},
    ],
    "device":i,
    "gender":"f",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "화장품/미용")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "스킨케어")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["화장품/미용"], axis=1, inplace=True)
df_all_b

In [ ]:
for i in devices:
    categories = [
          {"name": "남성화장품", "param": [ "50000202"]},
          {"name": "네일케어", "param": [ "50000196"]},
          {"name": "바디케어", "param": [ "50000197"]},
          {"name": "베이스메이크업", "param": [ "50000194"]},
          {"name": "뷰티소품", "param": [ "50000201"]},
          {"name": "색조메이크업", "param": [ "50000195"]},
          {"name": "선케어", "param": [ "50000191"]},
          {"name": "클렌징", "param": [ "50000192"]},
          {"name": "향수", "param": [ "50000200"]},
          {"name": "헤어케어", "param": [ "50000198"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "화장품/미용", "param": [ "50000002"]},
      {"name": "스킨케어", "param": [ "50000190"]},
    ],
    "device":i,
    "gender":"m",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "화장품/미용")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "스킨케어")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["화장품/미용"], axis=1, inplace=True)
df_all_c

In [ ]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)

## 엑셀에 저장
df_all.to_excel("MCLS_11_all_fm_all.xlsx", sheet_name="click ratio")
df_all.info()